# Импрт библиотек для первичного анализа

In [1]:
%pip install --upgrade matplotlib
%pip install --upgrade seaborn
%pip install --upgrade plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 3.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.8.2
    Uninstalling matplotlib-3.8.2:
      Successfully uninstalled matplotlib-3.8.2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 4.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: plotly
    Found existing installation: plotly 6.0.0
    Uninstalling plotly-6.0.0:
      Successfully uninstalled plotly-6.0.0
Note: you may need to restart the kernel to use updated packages.


In [21]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Загрузка данных

In [3]:
orders = pd.read_csv("orders.csv")
products = pd.read_csv("products.csv")
aisles = pd.read_csv("aisles.csv")
departments = pd.read_csv("departments.csv")
order_products_train = pd.read_csv("order_products__train.csv")
order_products_prior = pd.read_csv("order_products__prior.csv")

In [4]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [5]:
print(orders.dtypes)

order_id                    int64
user_id                     int64
eval_set                   object
order_number                int64
order_dow                   int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object


In [6]:
orders['days_since_prior_order'] = orders['days_since_prior_order'].fillna(0)
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [7]:
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [8]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [9]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [10]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


# Объединение данных в итоговую таблицу

In [11]:
user_order = pd.merge(orders, order_products_prior)
user_order.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,0.0,196,1,0
1,2539329,1,prior,1,2,8,0.0,14084,2,0
2,2539329,1,prior,1,2,8,0.0,12427,3,0
3,2539329,1,prior,1,2,8,0.0,26088,4,0
4,2539329,1,prior,1,2,8,0.0,26405,5,0


In [12]:
detailed_order = pd.merge(products, user_order, on='product_id', how='left')
detailed_order.head()

,product_id,product_name,aisle_id,department_id,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered
0,1,Chocolate Sandwich Cookies,61,19,3139998.0,138.0,prior,28.0,6.0,11.0,3.0,5.0,0.0
1,1,Chocolate Sandwich Cookies,61,19,1977647.0,138.0,prior,30.0,6.0,17.0,20.0,1.0,1.0
2,1,Chocolate Sandwich Cookies,61,19,389851.0,709.0,prior,2.0,0.0,21.0,6.0,20.0,0.0
3,1,Chocolate Sandwich Cookies,61,19,652770.0,764.0,prior,1.0,3.0,13.0,0.0,10.0,0.0
4,1,Chocolate Sandwich Cookies,61,19,1813452.0,764.0,prior,3.0,4.0,17.0,9.0,11.0,1.0


In [13]:
print(detailed_order.dtypes)
print(detailed_order.isna().sum())

product_id                  int64
product_name               object
aisle_id                    int64
department_id               int64
order_id                  float64
user_id                   float64
eval_set                   object
order_number              float64
order_dow                 float64
order_hour_of_day         float64
days_since_prior_order    float64
add_to_cart_order         float64
reordered                 float64
dtype: object
product_id                 0
product_name               0
aisle_id                   0
department_id              0
order_id                  11
user_id                   11
eval_set                  11
order_number              11
order_dow                 11
order_hour_of_day         11
days_since_prior_order    11
add_to_cart_order         11
reordered                 11
dtype: int64


In [14]:
detailed_order = pd.merge(detailed_order, departments, on='department_id', how='left')

detailed_order = pd.merge(detailed_order, aisles, on='aisle_id', how='left')

detailed_order.head()

,product_id,product_name,aisle_id,department_id,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,department,aisle
0,1,Chocolate Sandwich Cookies,61,19,3139998.0,138.0,prior,28.0,6.0,11.0,3.0,5.0,0.0,snacks,cookies cakes
1,1,Chocolate Sandwich Cookies,61,19,1977647.0,138.0,prior,30.0,6.0,17.0,20.0,1.0,1.0,snacks,cookies cakes
2,1,Chocolate Sandwich Cookies,61,19,389851.0,709.0,prior,2.0,0.0,21.0,6.0,20.0,0.0,snacks,cookies cakes
3,1,Chocolate Sandwich Cookies,61,19,652770.0,764.0,prior,1.0,3.0,13.0,0.0,10.0,0.0,snacks,cookies cakes
4,1,Chocolate Sandwich Cookies,61,19,1813452.0,764.0,prior,3.0,4.0,17.0,9.0,11.0,1.0,snacks,cookies cakes


In [15]:
print(detailed_order.dtypes)
print(detailed_order.isna().sum())

product_id                  int64
product_name               object
aisle_id                    int64
department_id               int64
order_id                  float64
user_id                   float64
eval_set                   object
order_number              float64
order_dow                 float64
order_hour_of_day         float64
days_since_prior_order    float64
add_to_cart_order         float64
reordered                 float64
department                 object
aisle                      object
dtype: object
product_id                 0
product_name               0
aisle_id                   0
department_id              0
order_id                  11
user_id                   11
eval_set                  11
order_number              11
order_dow                 11
order_hour_of_day         11
days_since_prior_order    11
add_to_cart_order         11
reordered                 11
department                 0
aisle                      0
dtype: int64


Выведем строки, где у нас есть пропуски и посмотрим на них

In [16]:
detailed_order[detailed_order.isna().any(axis=1)]

,product_id,product_name,aisle_id,department_id,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,department,aisle
1738025,3630,Protein Granola Apple Crisp,57,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,breakfast,granola
1768779,3718,Wasabi Cheddar Spreadable Cheese,21,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dairy eggs,packaged cheese
4043089,7045,Unpeeled Apricot Halves in Heavy Syrup,88,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pantry,spreads
16270114,25383,Chocolate Go Bites,61,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,snacks,cookies cakes
17805646,27499,Non-Dairy Coconut Seven Layer Bar,100,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing,missing
23483677,36233,Water With Electrolytes,100,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing,missing
24234095,37703,Ultra Sun Blossom Liquid 90 loads Fabric Enhan...,75,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,household,laundry
28038726,43725,Sweetart Jelly Beans,100,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing,missing
29660201,45971,12 Inch Taper Candle White,101,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,household,air fresheners candles
29931983,46625,Single Barrel Kentucky Straight Bourbon Whiskey,31,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,beverages,refrigerated


Получается, что у нас всего 11 строк, где есть пропуски в одних и тех же столбцах. С учетом размера нашего датасета мы можем их безболезнено удалить.

In [17]:
detailed_order = detailed_order.dropna()

In [18]:
detailed_order = detailed_order.reset_index(drop=True)
detailed_order.head(5)

,product_id,product_name,aisle_id,department_id,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,department,aisle
0,1,Chocolate Sandwich Cookies,61,19,3139998.0,138.0,prior,28.0,6.0,11.0,3.0,5.0,0.0,snacks,cookies cakes
1,1,Chocolate Sandwich Cookies,61,19,1977647.0,138.0,prior,30.0,6.0,17.0,20.0,1.0,1.0,snacks,cookies cakes
2,1,Chocolate Sandwich Cookies,61,19,389851.0,709.0,prior,2.0,0.0,21.0,6.0,20.0,0.0,snacks,cookies cakes
3,1,Chocolate Sandwich Cookies,61,19,652770.0,764.0,prior,1.0,3.0,13.0,0.0,10.0,0.0,snacks,cookies cakes
4,1,Chocolate Sandwich Cookies,61,19,1813452.0,764.0,prior,3.0,4.0,17.0,9.0,11.0,1.0,snacks,cookies cakes


# Обрезка данных

У нас получилось очень много данных, аж 33 млн строк. Из-за ограниченности вычислительных ресурсов, было принято решение ограничиться примерно 10000 строк.

In [29]:
np.random.seed(42)

min_orders = 5
active_users = (
    detailed_order.groupby("user_id")["order_number"]
      .count()[lambda s: s >= min_orders]
      .index
)

rng = np.random.default_rng(42)
sample_users = rng.choice(active_users, size=500, replace=False)


df = detailed_order[detailed_order["user_id"].isin(sample_users)]

df = (df.sort_values(["user_id", "order_number"])
                     .groupby("user_id")
                     .tail(200))

df

,product_id,product_name,aisle_id,department_id,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,department,aisle
9724967,16617,Organic Muenster Cheese Slices,21,16,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,8.0,0.0,dairy eggs,packaged cheese
12882526,21267,Sourdough Bread,112,3,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,3.0,0.0,bakery,bread
15261391,24561,Organic Cheese Frozen Pizza,79,1,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,1.0,0.0,frozen,frozen pizza
15463034,24852,Banana,24,4,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,2.0,0.0,produce,fresh fruits
17699822,27344,Uncured Genoa Salami,96,20,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,6.0,0.0,deli,lunch meat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5576077,9434,Bag of Large Lemons,34,1,444424.0,206100.0,prior,1.0,3.0,13.0,0.0,1.0,0.0,frozen,frozen meat seafood
25649802,39922,Celery Sticks,123,4,444424.0,206100.0,prior,1.0,3.0,13.0,0.0,2.0,0.0,produce,packaged vegetables fruits
5576078,9434,Bag of Large Lemons,34,1,2483012.0,206100.0,prior,2.0,6.0,16.0,30.0,2.0,1.0,frozen,frozen meat seafood
17589949,27156,Organic Black Beans,59,15,2483012.0,206100.0,prior,2.0,6.0,16.0,30.0,1.0,0.0,canned goods,canned meals beans


In [30]:
df.to_csv('InstaCart.csv', index=False)